# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 1e7 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "battle1" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 4 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 409600 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 128 # Number of units in hidden layer.
batch_size = 5120 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Academy' started successfully!


Unity Academy name: Academy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: KnightBrainInternal
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 65
        Action space type: continuous
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: , 


### Train the Agent(s)

In [ ]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/ppo/model-6000000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo/model-6000000.cptk


Saved Model
Step: 6010000. Mean Reward: 0.16079630521882052. Std of Reward: 0.8739746306772338.
Step: 6020000. Mean Reward: 0.14588699178233794. Std of Reward: 0.9233279231520125.
Step: 6030000. Mean Reward: 0.15530362023092947. Std of Reward: 0.9592444747800462.
Step: 6040000. Mean Reward: 0.11697882976362385. Std of Reward: 0.9567132711368739.
Step: 6050000. Mean Reward: 0.13955797011730867. Std of Reward: 0.9464953381694872.
Saved Model
Step: 6060000. Mean Reward: 0.11657153806726021. Std of Reward: 0.9445019402198195.
Step: 6070000. Mean Reward: 0.13973794727642613. Std of Reward: 0.9353966526991485.
Step: 6080000. Mean Reward: 0.16969541587038584. Std of Reward: 0.9383504840932547.
Step: 6090000. Mean Reward: 0.22572973501245977. Std of Reward: 0.9457834137138451.
Step: 6100000. Mean Reward: 0.19010421390634102. Std of Reward: 0.9338804642696767.
Saved Model
Step: 6110000. Mean Reward: 0.07625486080244485. Std of Reward: 0.9332467751424411.
Step: 6120000. Mean Reward: 0.0819806366

Step: 6970000. Mean Reward: 0.25694827798238534. Std of Reward: 0.9339327526308899.
Step: 6980000. Mean Reward: 0.23759862036470683. Std of Reward: 0.9270328024054929.
Step: 6990000. Mean Reward: 0.21157020800540924. Std of Reward: 0.9540913019422116.
Step: 7000000. Mean Reward: 0.2338967615072466. Std of Reward: 0.9455585161094274.
Saved Model
Step: 7010000. Mean Reward: 0.2301190520389679. Std of Reward: 0.9415788809772445.
Step: 7020000. Mean Reward: 0.2325166652082551. Std of Reward: 0.9465079824768358.
Step: 7030000. Mean Reward: 0.24705904781980437. Std of Reward: 0.9658506771340248.
Step: 7040000. Mean Reward: 0.20232636720034208. Std of Reward: 0.9528648135952799.
Step: 7050000. Mean Reward: 0.23612785897434138. Std of Reward: 0.9775233304196529.
Saved Model
Step: 7060000. Mean Reward: 0.2232585904783533. Std of Reward: 0.9566761125638432.
Step: 7070000. Mean Reward: 0.21230152568916097. Std of Reward: 0.9524605013076778.
Step: 7080000. Mean Reward: 0.19031903015591423. Std of 

Step: 7930000. Mean Reward: 0.031050327329081848. Std of Reward: 0.8193390349661247.
Step: 7940000. Mean Reward: 0.02915615835675506. Std of Reward: 0.7700987151993515.
Step: 7950000. Mean Reward: 0.08198024549122733. Std of Reward: 0.7947092102908042.
Saved Model
Step: 7960000. Mean Reward: 0.06611764470634617. Std of Reward: 0.768662120165768.
Step: 7970000. Mean Reward: 0.08216241866872113. Std of Reward: 0.8044712022920306.
Step: 7980000. Mean Reward: 0.05554139721007139. Std of Reward: 0.7664065754503651.
Step: 7990000. Mean Reward: 0.06585490164039702. Std of Reward: 0.8169401797543675.
Step: 8000000. Mean Reward: 0.04770186709286286. Std of Reward: 0.7999964362931478.
Saved Model
Step: 8010000. Mean Reward: 0.09745105879776081. Std of Reward: 0.8394342443423812.
Step: 8020000. Mean Reward: 0.06430711017858401. Std of Reward: 0.7950951229549679.
Step: 8030000. Mean Reward: 0.1157247781481688. Std of Reward: 0.8521563246188882.
Step: 8040000. Mean Reward: 0.10541046292819212. Std 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo/model-50001.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo/model-50001.cptk


KeyboardInterrupt: 